# DuckDB + Parquet Data Exploration Template

In [ ]:
from pathlib import Path
from tqdm import tqdm
import json
from IPython.display import display

import duckdb
import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib import colors, ticker
from typing import Optional, Tuple
from matplotlib.cm import ScalarMappable
import folium
import matplotlib.dates as mdates
from random import shuffle

from src.gen_points_map import compute_step, make_equal_area_hex_grid, make_equal_area_grid
from src.geo_util import assign_intersection_id
from src.plotting.util import plot_gdf_column, load_grids

import warnings
warnings.filterwarnings("ignore")  # hide every warning

In [ ]:
from PIL import Image

image_paths = sorted(list(Path("/Users/kyledorman/data/planet_coverage/figs/points_30km/days_with_sample/").glob("max_*valid*")))[:-1]

def make_gif(image_paths):
    frames = [Image.open(image) for image in image_paths]
    frame_one = frames[0]
    frame_one.save("/Users/kyledorman/data/planet_coverage/figs/points_30km/days_with_sample/max_days_with_sample_valid.gif", format="GIF", append_images=frames,
               save_all=True, duration=700, loop=0)
    
make_gif(image_paths)

# --- Configuration ---

In [ ]:
BASE = Path("/Users/kyledorman/data/planet_coverage/points_30km/")  # <-- update this

# Example path patterns
f_pattern = "*/coastal_results/*/*/*/coastal_points.parquet"
all_files_pattern = str(BASE / f_pattern)

# Combined list used later when we search individual files
all_parquets = list(BASE.glob(f_pattern))

FILE = all_parquets[0]

len(all_parquets)

In [ ]:
query_df, grids_df, hex_grid = load_grids(BASE.parent / "shorelines")

MIN_DIST = 20.0
valid = ~grids_df.is_land & (grids_df.dist_km.isna() | (grids_df.dist_km < MIN_DIST))
grids_df = grids_df[valid].copy()

local_crop = gpd.read_file(BASE.parent / "shorelines" / "la.geojson")

# Filter grids to CA region
query_local = query_df[query_df.geometry.intersects(local_crop.union_all())]
grids_local = grids_df[grids_df.geometry.intersects(query_local.union_all())]
hex_grid_local = hex_grid[hex_grid.geometry.intersects(query_local.union_all())]

In [ ]:
# --- Connect to DuckDB ---
con = duckdb.connect()

In [ ]:
# Register a view for all files
con.execute(
    f"""
    CREATE OR REPLACE VIEW samples_all AS
    SELECT * FROM read_parquet('{all_files_pattern}');
"""
)

In [ ]:
# Register a view for a single file for faster iteration
con.execute(f"""
    CREATE OR REPLACE VIEW samples_one AS
    SELECT * FROM '{FILE}'
""")

In [ ]:
# --- Schema Inspection ---
print("Schema of samples_one:")
df = con.execute("DESCRIBE samples_one").fetchdf()
print(df)

In [ ]:
# --- Preview Data ---
df_preview = con.execute("SELECT * FROM samples_one LIMIT 5").fetchdf()
df_preview

In [ ]:
# --- Filter By Time ---
df_2024 = con.execute("""
    SELECT *
    FROM samples_one
    WHERE acquired >= '2024-01-01' AND acquired < '2025-01-01'
    LIMIT 100
""").fetchdf()
df_2024.head(5)

In [ ]:
# --- Filter By Time ---
df_2024 = con.execute("""
    SELECT MIN(acquired) as first_sample,
    FROM samples_all
""").fetchdf()
df_2024

In [ ]:
df_dates = con.execute("""
    SELECT 
        grid_id, 
        cell_id,
        MIN(acquired) as first_sample,
        MAX(acquired) as last_sample
    FROM samples_all
    WHERE item_type = 'PSScene'
    GROUP BY grid_id, cell_id
""").fetchdf().set_index("grid_id")

print(df_dates.first_sample.min(), df_dates.first_sample.max())
print(df_dates.last_sample.min(), df_dates.last_sample.max())
print(len(df_dates), len(grids_df))
print(len(df_dates.cell_id.unique()), len(query_df))

In [ ]:
# --- Load Geo Points and Join ---

# Sample count per grid cell
df_counts = con.execute("""
    SELECT grid_id, COUNT(*) as sample_count
    FROM samples_all
    WHERE item_type = 'PSScene'
    GROUP BY grid_id
""").fetchdf().set_index("grid_id")

geo_plot = grids_ca.join(df_counts, how="left").fillna({"sample_count": 0})

plot_gdf_column(geo_plot, "sample_count", title="Sample Count PSScene", show_coastlines=True)

In [ ]:
# --- Load Geo Points and Join ---

# Sample count per grid cell
df_counts = con.execute("""
    SELECT grid_id, COUNT(*) as sample_count
    FROM samples_all
    WHERE item_type = 'PSScene'
    GROUP BY grid_id
""").fetchdf().set_index("grid_id")

geo_plot = grids_ca.join(df_counts, how="left").fillna({"sample_count": 0})

df = pd.DataFrame(geo_plot.groupby("hex_id")["sample_count"].sum())
df = df[df.index >= 0]
df = df.join(hex_grid_ca[["geometry"]])
gdf = gpd.GeoDataFrame(df, geometry="geometry")

plot_gdf_column(gdf, "sample_count", title="Sample Count PSScene Hex", show_coastlines=True)

In [ ]:
df = pd.DataFrame(grids_df.groupby("hex_id")["dist_km"].median())
df = df[df.index >= 0]
df = df.join(hex_grid[["geometry"]])
gdf = gpd.GeoDataFrame(df, geometry="geometry")

plot_gdf_column(gdf, "dist_km", title="dist_km", show_coastlines=True)

In [ ]:
# --- Load Geo Points and Join ---

# Sample count per grid cell
df_counts = con.execute("""
    SELECT grid_id, COUNT(*) as sample_count
    FROM samples_all
    WHERE item_type = 'PSScene'
    AND coverage_pct > 0.5
    AND publishing_stage = 'finalized'
    AND quality_category = 'standard'
    AND has_sr_asset
    AND ground_control
    GROUP BY grid_id
""").fetchdf().set_index("grid_id")

geo_plot = grids_ca.join(df_counts, how="left").fillna({"sample_count": 0})

df = pd.DataFrame(geo_plot.groupby("hex_id")["sample_count"].sum())
df = df[df.index >= 0]
df = df.join(hex_grid_ca[["geometry"]])
gdf = gpd.GeoDataFrame(df, geometry="geometry")

plot_gdf_column(gdf, "sample_count", title="High Quality Sample Count", show_coastlines=True)

In [ ]:
# --- Load Geo Points and Join ---

df = con.execute(
    """
    SELECT grid_id, SUM(coverage_pct > 0.5)::DOUBLE AS coverage_count,  COUNT(*) as sample_count
    FROM samples_all
    WHERE item_type = 'PSScene'
    GROUP BY grid_id
"""
).fetchdf().set_index("grid_id")

geo_coverage = grids_ca.join(df, how="left").fillna({"sample_count": 1.0, "coverage_count": 0.0})
geo_coverage["sample_pct"] = geo_coverage.coverage_count / geo_coverage.sample_count 

plot_gdf_column(geo_coverage, "sample_pct", title="% Greater than 50% coverage", show_coastlines=True)

agg = (
    geo_coverage.groupby('hex_id', as_index=False, sort=False)[['coverage_count', 'sample_count']]
      .sum()                                # ← sums within each hex
      .assign(
          coverage_pct=lambda d: d['coverage_count'] / d['sample_count']  # or * 100 for %
      )
)
agg = agg.set_index("hex_id")[agg.index >= 0]
agg = agg.join(hex_grid_ca[["geometry"]])
gdf = gpd.GeoDataFrame(agg, geometry="geometry")

plot_gdf_column(gdf, "coverage_pct", title="% Greater than 50% coverage", show_coastlines=True)

In [ ]:
# --- Load Geo Points and Join ---

df_pct = con.execute(
    """
    SELECT grid_id,
           SUM(intersects_grid_centroid)::DOUBLE as coverage_count,
           COUNT(*) AS sample_count
    FROM samples_all
    WHERE item_type = 'PSScene'
    GROUP BY grid_id
"""
).fetchdf().set_index("grid_id")

geo_coverage = grids_ca.join(df_pct, how="left").fillna({"sample_count": 1.0, "coverage_count": 0.0})
geo_coverage["sample_pct"] = geo_coverage.coverage_count / geo_coverage.sample_count 

plot_gdf_column(geo_coverage, "sample_pct", title="% Intersects Grid Center", show_coastlines=True)

agg = (
    geo_coverage.groupby('hex_id', as_index=False, sort=False)[['coverage_count', 'sample_count']]
      .sum()                                # ← sums within each hex
      .assign(
          coverage_pct=lambda d: d['coverage_count'] / d['sample_count']  # or * 100 for %
      )
)
agg = agg.set_index("hex_id")[agg.index >= 0]
agg = agg.join(hex_grid_ca[["geometry"]])
gdf = gpd.GeoDataFrame(agg, geometry="geometry")

plot_gdf_column(gdf, "coverage_pct", title="% Intersects Grid Center", show_coastlines=True)

In [ ]:
# --- Load Geo Points and Join ---

df_pct = con.execute(
    """
    SELECT grid_id,
           SUM(intersects_grid_centroid)::DOUBLE  / SUM(coverage_pct > 0.5)::DOUBLE AS frac_coverage
    FROM samples_all
    WHERE item_type = 'PSScene'
    GROUP BY grid_id
"""
).fetchdf().set_index("grid_id")

geo_pct = grids_ca.join(df_pct, how="left").fillna({"frac_coverage": 0.0})

plot_gdf_column(geo_pct, "frac_coverage", title="Ration grid center vs 50% coverage", show_coastlines=True)

In [ ]:
# --- Load Geo Points and Join ---

df_pct = con.execute(
    """
    SELECT grid_id,
           SUM(intersects_grid_centroid)::DOUBLE  / SUM(coverage_pct > 0.75)::DOUBLE AS frac_coverage
    FROM samples_all
    WHERE item_type = 'PSScene'
    GROUP BY grid_id
"""
).fetchdf().set_index("grid_id")

geo_pct = grids_ca.join(df_pct, how="left").fillna({"frac_coverage": 0.0})

plot_gdf_column(geo_pct, "frac_coverage", title="Ration grid center vs 75% coverage", show_coastlines=True)

In [ ]:
df_cov = con.execute("""
    SELECT
        grid_id,
        COUNT(*)                                                  AS sample_count,
        SUM(CASE WHEN coverage_pct > 0.5 THEN 1 ELSE 0 END)      AS coverage_count,
    FROM samples_all
    WHERE item_type        = 'PSScene'
      AND publishing_stage = 'finalized'
      AND quality_category = 'standard'
      AND has_sr_asset
      AND ground_control
    GROUP BY grid_id
""").fetchdf().set_index("grid_id")

# join with your coastal grid GeoDataFrame
geo_coverage = grids_ca.join(df_cov, how="left").fillna({"sample_count": 1.0, "coverage_count": 0.0})
geo_coverage["sample_pct"] = geo_coverage.coverage_count / geo_coverage.sample_count 

plot_gdf_column(geo_coverage, "sample_pct", title="% of high quality captures with >50 % area", show_coastlines=True)

agg = (
    geo_coverage.groupby('hex_id', as_index=False, sort=False)[['coverage_count', 'sample_count']]
      .sum()                                # ← sums within each hex
      .assign(
          coverage_pct=lambda d: d['coverage_count'] / d['sample_count']  # or * 100 for %
      )
)
agg = agg.set_index("hex_id")[agg.index >= 0]
agg = agg.join(hex_grid_ca[["geometry"]])
gdf = gpd.GeoDataFrame(agg, geometry="geometry")

plot_gdf_column(gdf, "coverage_pct", title="% of high quality captures with >50 % area", show_coastlines=True)

In [ ]:
# ---- Histogram Per Day Counts (w/Publish Stage) ---- #

# 1. Pull per-day counts broken out by stage
df_stage = con.execute("""
    SELECT
        date_trunc('day', acquired) AS day,
        publishing_stage,
        COUNT(DISTINCT id) AS cnt
    FROM samples_all
    WHERE item_type = 'PSScene'
    GROUP BY day, publishing_stage
    ORDER BY day
""").fetchdf()

# 2. Normalize day column and pivot so each stage is its own column
df_stage['day'] = pd.to_datetime(df_stage['day']).dt.date
df_pivot = (
    df_stage
    .pivot(index='day', columns='publishing_stage', values='cnt')
    .fillna(0)
)

# 3. Ensure a consistent stack order
stages = ['preview', 'finalized', 'standard']
df_pivot = df_pivot.reindex(columns=stages, fill_value=0)

# 4. Plot stacked bars
fig, ax = plt.subplots(figsize=(10, 4))
bottom = np.zeros(len(df_pivot))

for stage in stages:
    ax.bar(
        df_pivot.index.astype(str),
        df_pivot[stage],
        bottom=bottom,
        label=stage
    )
    bottom += df_pivot[stage].values

ax.set_xticklabels(df_pivot.index.astype(str), rotation=45, ha='right')
ax.set_title("Sample Count per Day by Publishing Stage")
ax.set_xlabel("Day")
ax.set_ylabel("Sample Count")
ax.legend(title="Publishing Stage")
fig.tight_layout()
plt.show()

In [ ]:
def compute_minmax(column: str) -> pd.DataFrame:
    sql = f"""
        SELECT
        MIN({column}) AS minv,
        MAX({column}) AS maxv
        FROM samples_all
        WHERE item_type = 'PSScene'
    """
    return con.execute(sql).fetchdf()

def compute_histogram(column: str, nbins: int = 30) -> pd.DataFrame:
    """
    Runs DuckDB's histogram() table function on `column` in samples_all (filtered to PSScene)
    and returns a DataFrame with columns: bin_upper, frequency.
    """
    sql = f"""
        WITH bounds AS (
          SELECT
            MIN({column}) AS mn,
            MAX({column}) AS mx
          FROM samples_all
          WHERE item_type = 'PSScene'
        )
        SELECT
          -- histogram() returns a MAP<upper_boundary, count>
          histogram(
            {column},
            equi_width_bins(bounds.mn::DOUBLE, bounds.mx::DOUBLE, {nbins}::BIGINT, True)
          ) AS hist_map
        FROM samples_all
        CROSS JOIN bounds
        WHERE item_type = 'PSScene';
    """
    hist_map = con.execute(sql).fetchdf().iloc[0]["hist_map"]

    
    # Unpack into a two-column DataFrame
    df = pd.DataFrame({
        'bin_upper': list(hist_map.keys()),
        'count':     list(hist_map.values())
    })
    df = df.sort_values('bin_upper').reset_index(drop=True)
    uppers = df['bin_upper'].tolist()
    bin_size = uppers[1] - uppers[0]
    # Compute lower edge from previous upper
    lowest = uppers[0] - bin_size
    lowers = [lowest] + uppers[:-1]
    df["bin_lower"] = pd.Series(lowers)
    df["centers"] = (df["bin_lower"] + df['bin_upper']) / 2
    df["widths"]  = df['bin_upper'] - df["bin_lower"]
    return df

In [ ]:
# Set up a 2×2 grid of histograms
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()


# 2. Plotting all four angle columns
for ax, col in zip(axes, ["satellite_azimuth", "sun_azimuth", "sun_elevation", "view_angle"]):
    df_hist = compute_histogram(col, nbins=30)

    ax.bar(df_hist["centers"], df_hist['count'], width=df_hist["widths"] * 0.9)
    ax.set_title(f"Histogram of {col.replace('_',' ').title()}")
    ax.set_xlabel(col.replace('_',' ').title())
    ax.set_ylabel("Frequency")
    
fig.tight_layout()
plt.show()

In [ ]:
# --- Histogram of Sample Count per Month ---
df_monthly = con.execute("""
    SELECT date_trunc('month', acquired) AS month, COUNT(DISTINCT id) AS sample_count
    FROM samples_all
    WHERE item_type = 'PSScene'
    GROUP BY month
    ORDER BY month
""").fetchdf()

# Plot histogram
plt.figure(figsize=(10,4))
plt.bar(df_monthly['month'].astype(str), df_monthly['sample_count'])
plt.xticks(rotation=45, ha='right')
plt.title("Sample Count per Month (Unique Scenes)")
plt.xlabel("Month")
plt.ylabel("Sample Count")
plt.tight_layout()
plt.show()

In [ ]:
# --- Histogram of coverage_pct ---
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

df_hist = compute_histogram("coverage_pct", nbins=10)

ax.bar(df_hist["centers"], df_hist['count'], width=df_hist["widths"] * 0.9)
ax.set_title(f"Histogram of {'coverage_pct'.title()}")
ax.set_xlabel("coverage_pct".title())
ax.set_ylabel("Frequency")
    
fig.tight_layout()
plt.show()

In [ ]:
# --- Histogram of clear_percent ---
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

df_hist = compute_histogram("clear_percent", nbins=30)

ax.bar(df_hist["centers"], df_hist['count'], width=df_hist["widths"] * 0.9)
ax.set_title(f"Histogram of {'clear_percent'.title()}")
ax.set_xlabel("clear_percent".title())
ax.set_ylabel("Frequency")
    
fig.tight_layout()
plt.show()

In [ ]:
query = """
SELECT
    grid_id,
    MIN(tide_height) AS obs_min_tide_height,
    MAX(tide_height) AS obs_max_tide_height
FROM samples_all
WHERE
    acquired >= TIMESTAMP '2023-12-01'
    AND acquired <  TIMESTAMP '2025-01-01'
    AND item_type        = 'PSScene'
    AND publishing_stage = 'finalized'
    AND quality_category = 'standard'
    AND has_sr_asset
    AND ground_control
    AND has_tide_data
GROUP BY grid_id
ORDER BY grid_id;
"""

df = con.execute(query).fetchdf().set_index("grid_id")

geo_tide = grids_ca.join(df, how="left").dropna(subset=["obs_min_tide_height", "obs_max_tide_height", 'tide_range'])
geo_tide['obs_tide_range'] = geo_tide.obs_max_tide_height - geo_tide.obs_min_tide_height
geo_tide['phase_coverage'] = geo_tide['obs_tide_range'] / geo_tide['tide_range']
plot_gdf_column(geo_tide, "phase_coverage", title="phase_coverage", show_coastlines=True)

hex_tide = (
    geo_tide
      .groupby("hex_id")
      .agg(                 # keep one row per hex_id
          obs_min_tide_height=("obs_min_tide_height", "min"),   # lowest observed tide
          obs_max_tide_height=("obs_max_tide_height", "max"),   # highest observed tide
          tide_min=("tide_min", "min"),   # lowest tide
          tide_max=("tide_max", "max"),   # highest tide
      )
)
hex_tide['tide_range'] = hex_tide.tide_max - hex_tide.tide_min
hex_tide['obs_tide_range'] = hex_tide.obs_max_tide_height - hex_tide.obs_min_tide_height
hex_tide['phase_coverage'] = hex_tide.obs_tide_range / hex_tide.tide_range

hex_tide = hex_tide[hex_tide.index >= 0]
hex_tide = hex_tide.join(hex_grid_ca[["geometry"]])
gdf = gpd.GeoDataFrame(hex_tide, geometry="geometry")

plot_gdf_column(gdf, "phase_coverage", title="phase_coverage", show_coastlines=True)

In [ ]:
query = """
-- one row per grid_id × calendar-month
SELECT
    grid_id,
    /* month_start = first day of the month, keeps it sortable & readable */
    DATE_TRUNC('month', acquired) AS month_start,
    COUNT(*)                       AS sample_count,
    COUNT_IF(has_8_channel)        AS count_8_channel      -- rows where flag = TRUE
FROM samples_all
WHERE
    item_type        = 'PSScene'
    AND coverage_pct > 0.5
GROUP BY grid_id, month_start
ORDER BY grid_id, month_start;
"""

monthly_counts = con.execute(query).fetchdf().set_index("grid_id")
monthly_counts["pct_8_channel"] = monthly_counts.count_8_channel / monthly_counts.sample_count
monthly_counts

In [ ]:
monthly_counts.loc[12487145].plot.scatter(y="pct_8_channel", x="month_start")

In [ ]:
monthly_counts.loc[12487145].plot.scatter(y="sample_count", x="month_start")

In [ ]:
first_month_8_channel = monthly_counts[monthly_counts.pct_8_channel > 0.5].reset_index().drop_duplicates(subset=["grid_id"]).set_index("grid_id")
first_month_8_channel

In [ ]:
geo_tide = grids_ca.join(first_month_8_channel[["month_start"]], how="left").dropna(subset=['month_start'])
plot_gdf_column(geo_tide, "month_start", title="month_start", show_coastlines=True)

In [ ]:
hex_counts = grids_ca[["hex_id"]].join(monthly_counts, how='left')

agg = (
    hex_counts.groupby(['hex_id', 'month_start'], as_index=False, sort=False)[['count_8_channel', 'sample_count']]
      .sum()                                # ← sums within each hex
      .assign(
          pct_8_channel=lambda d: d['count_8_channel'] / d['sample_count']  # or * 100 for %
      )
)

agg = agg[agg.index >= 0]
agg = agg[agg.pct_8_channel > 0.5].reset_index().drop_duplicates(subset=["hex_id"]).set_index("hex_id")
agg = agg.join(hex_grid_ca[["geometry"]])
gdf = gpd.GeoDataFrame(agg, geometry="geometry")

plot_gdf_column(gdf, "month_start", title="month_start", show_coastlines=True)